In [31]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\gen_data.parquet')

In [33]:
df['C_GIOITINH'] = df['C_GIOITINH'].replace('O', 'M')

In [34]:
train = df[df['SAMPLE_TYPE'] == 'TRAIN']
oos = df[df['SAMPLE_TYPE'] == 'OOS']
oot = df[df['SAMPLE_TYPE'] == 'OOT']

In [35]:
y_train = train['BAD_NEXT_12M']
X_train = train.drop('BAD_NEXT_12M', axis=1)
y_oos = oos['BAD_NEXT_12M']
X_oos = oos.drop('BAD_NEXT_12M', axis=1)
y_oot = oot['BAD_NEXT_12M']
X_oot = oot.drop('BAD_NEXT_12M', axis=1)
# 1. Chỉ chọn các biến số (Numeric) để tính toán
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
corr_matrix = X_train[numeric_cols].corr().abs() # Lấy trị tuyệt đối (âm hay dương đều là tương quan mạnh)

# 2. Chọn ngưỡng cắt (Threshold)
# Với EBM, ngưỡng an toàn là 0.7. Ngưỡng 0.8 là bắt buộc phải xử lý.
threshold = 0.7

# 3. Lọc ra các cặp biến vượt ngưỡng
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Tạo danh sách chi tiết để bạn dễ quyết định giữ ai/bỏ ai
high_corr_pairs = []
for col in upper.columns:
    high_cols = upper.index[upper[col] > threshold].tolist()
    for row in high_cols:
        val = upper.loc[row, col]
        high_corr_pairs.append((row, col, val))

# Sắp xếp theo độ tương quan giảm dần
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 4. In kết quả
print(f"Tìm thấy {len(high_corr_pairs)} cặp biến tương quan cao (> {threshold}):")
print("-" * 60)
print(f"{'Biến 1 (Giữ?)':<30} | {'Biến 2 (Bỏ?)':<30} | {'Corr':<10}")
print("-" * 60)
for pair in high_corr_pairs:
    print(f"{pair[0]:<30} | {pair[1]:<30} | {pair[2]:.4f}")

Tìm thấy 104 cặp biến tương quan cao (> 0.7):
------------------------------------------------------------
Biến 1 (Giữ?)                  | Biến 2 (Bỏ?)                   | Corr      
------------------------------------------------------------
AFLIMT_MIN                     | AFLIMT_AVG                     | 1.0000
N_AVG_DEPOSIT_12M              | N_AVG_CD_12M                   | 1.0000
N_AVG_DEPOSIT_6M               | N_AVG_CD_12M                   | 1.0000
N_AVG_DD_12M                   | N_AVG_CD_12M                   | 1.0000
N_AVG_DEPOSIT_12M              | N_AVG_DD_12M                   | 1.0000
N_AVG_DEPOSIT_6M               | N_AVG_DD_12M                   | 1.0000
AFLIMT_MAX                     | AFLIMT_AVG                     | 1.0000
N_AVG_DEPOSIT_12M              | N_AVG_DEPOSIT_6M               | 1.0000
HAS_SHORTTERM_LOAN             | HAS_LONGTERM_LOAN              | 1.0000
PURCOD_MAX                     | PURCOD_MIN                     | 1.0000
MAX_DPD_12M              

In [ ]:
vars_to_drop = [
    # Deposit
    "N_AVG_DEPOSIT_6M",
    "N_AVG_DD_12M",
    "N_AVG_CD_12M",

    # Balance
    "CBALORG",
    "CBAL_MIN",
    "CBAL_MAX",
    "CBAL_AVG",

    # Limit
    "AFLIMT_MIN",
    "AFLIMT_MAX",

    # Overdue / DPD
    "MAX_DPD_12M_OBS",
    "SUM_ALL_OD_12M",

    # Income ratios
    "CBAL_TO_INC_12MON",
    "PTI_RATIO",

    # Collateral / House
    "SOHUUNHA",
    "COLLATERAL_VALUE",

    # Flag variables (trùng với biến gốc)
    "FLAG_CASH_ADVANCE",

    # Education (trùng với income)
    "TRINHDO",

    "HAS_SHORTTERM_LOAN",       # Trùng với HAS_LONGTERM
    "PURCOD_MIN", 
    "REAL_GDP_GROWTH_12M"
]

X_train.drop(vars_to_drop, axis=1,inplace=True)
X_oos.drop(vars_to_drop, axis=1,inplace=True)
X_oot.drop(vars_to_drop, axis=1,inplace=True)

In [37]:
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
corr_matrix = X_train[numeric_cols].corr().abs() # Lấy trị tuyệt đối (âm hay dương đều là tương quan mạnh)

# 2. Chọn ngưỡng cắt (Threshold)
# Với EBM, ngưỡng an toàn là 0.7. Ngưỡng 0.8 là bắt buộc phải xử lý.
threshold = 0.7

# 3. Lọc ra các cặp biến vượt ngưỡng
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

# Tạo danh sách chi tiết để bạn dễ quyết định giữ ai/bỏ ai
high_corr_pairs = []
for col in upper.columns:
    high_cols = upper.index[upper[col] > threshold].tolist()
    for row in high_cols:
        val = upper.loc[row, col]
        high_corr_pairs.append((row, col, val))

# Sắp xếp theo độ tương quan giảm dần
high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

# 4. In kết quả
print(f"Tìm thấy {len(high_corr_pairs)} cặp biến tương quan cao (> {threshold}):")
print("-" * 60)
print(f"{'Biến 1 (Giữ?)':<30} | {'Biến 2 (Bỏ?)':<30} | {'Corr':<10}")
print("-" * 60)
for pair in high_corr_pairs:
    print(f"{pair[0]:<30} | {pair[1]:<30} | {pair[2]:.4f}")

Tìm thấy 19 cặp biến tương quan cao (> 0.7):
------------------------------------------------------------
Biến 1 (Giữ?)                  | Biến 2 (Bỏ?)                   | Corr      
------------------------------------------------------------
UTILIZATION_RATE               | DTI_RATIO                      | 0.9053
INCOME                         | AFLIMT_AVG                     | 0.8894
BAD_CURRENT                    | MAX_NHOMNOCIC                  | 0.8841
INHERENT_RISK                  | CURRENT_RISK                   | 0.8729
MAX_DPD_12M                    | MAX_NHOMNOCIC                  | 0.8712
CURRENT_RISK                   | RATE_AVG                       | 0.8386
CBAL                           | N_AVG_OVERDUE_CBAL_12M         | 0.8101
DURATION_MAX                   | REMAINING_DURATION_MAX         | 0.8055
CURRENT_RISK                   | CNT_MIN_PAY_6M                 | 0.7985
CBAL                           | DTI_RATIO                      | 0.7920
BASE_AUM                  

In [38]:
X_train['BAD_NEXT_12M'] = y_train
X_oos['BAD_NEXT_12M'] = y_oos
X_oot['BAD_NEXT_12M'] = y_oot

df_merged = pd.concat([X_train, X_oos, X_oot], axis=0, ignore_index=True)
df_merged.to_parquet('dataset.parquet', index=False)